# Google Colab-related code

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

import sys
sys.path.append('/content/drive/My Drive/ai_image_classification_cifar/code')
%cd /content/drive/My\ Drive/ai_image_classification_cifar/code

Mounted at /content/drive
/content/drive/My Drive/ai_image_classification_cifar/code


# Set up environment

In [2]:
#Install necessary dependencies
#!bash install-dependencies.sh

#Install packages
from required_packages import *

# Source images

In [3]:
#Run this code when sourcing images from Kaggle account. However, do not run this when images are already sourced.
class Images:
    '''
    A class to source images.
    '''
    def __init__(self, num_images):
        self.num_images = num_images
        self.orig_dir = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images'
        self.dest_dir = '/kaggle/working/cifake'

    def copy_images(self):
        categories = ['FAKE', 'REAL']
        dataset_type = ['train', 'test']

        #Copy train & test images
        for i in dataset_type:
            for j in categories:
                orig_dir = os.path.join(self.orig_dir, i, j)
                dest_dir = os.path.join(self.dest_dir, i, j)
                functions.source_images(orig_dir = orig_dir, dest_dir = dest_dir, num_images = self.num_images, seed = 23)
        #Copy validation images
        for j in categories:
            train_dir= os.path.join(self.dest_dir, 'train', j)
            validation_dir = '/kaggle/working/cifake/validation'

            all_files = os.listdir(train_dir)
            random.seed(23)
            selected_files = random.sample(all_files, 100)

            for file in selected_files:
                train_file_path = os.path.join(train_dir, file)
                validation_file_path = os.path.join(validation_dir, j, file)
                os.makedirs(validation_file_path, exist_ok=True)
                shutil.copy(train_file_path, validation_file_path)

                os.remove(train_file_path)

# Preprocess images

In [4]:
class Preprocess:
    def __init__(self, **mdict):
        self.mdict = mdict

    def create_generators(self):
        train_datagen = ImageDataGenerator(
            rescale = self.mdict['generators']['rescale'],
            rotation_range = self.mdict['generators']['rotation_range'],
            width_shift_range = self.mdict['generators']['width_shift_range'],
            height_shift_range = self.mdict['generators']['height_shift_range'],
            shear_range = self.mdict['generators']['shear_range'],
            zoom_range = self.mdict['generators']['zoom_range'],
            fill_mode = self.mdict['generators']['fill_mode'])

        train_generator = train_datagen.flow_from_directory(
            self.mdict['info']['train_dir'],
            target_size = (224, 224),
            batch_size = 32,
            classes = self.mdict['info']['classes'],
            class_mode = 'binary')

        validation_generator = ImageDataGenerator().flow_from_directory(
            self.mdict['info']['validation_dir'],
            target_size = (224, 224),
            batch_size = 32,
            classes = self.mdict['info']['classes'],
            class_mode = 'binary')

        test_generator = ImageDataGenerator().flow_from_directory(
            self.mdict['info']['test_dir'],
            target_size = (224, 224),
            batch_size = 32,
            classes = self.mdict['info']['classes'],
            shuffle = False)

        return train_generator, validation_generator, test_generator

# Load YAML file and create training, validation, and test datasets

In [5]:
!python create_yaml_files.py

Dictionaries updated and saved.


# Transfer learning from pre-trained MobileNet V2 Model (Google)


In [86]:
import tensorflow as tf
import pickle

class TransferLearning:
    def __init__(self, train_generator, validation_generator, **mdict):  # Added validation_generator
        self.mdict = mdict
        self.train_generator = train_generator
        self.validation_generator = validation_generator  # Added validation_generator

    def create_base_model(self):
        IMG_SIZE = tuple(self.mdict['preprocess']['resize'])
        IMG_SHAPE = IMG_SIZE + (3,)
        # Ensure the method name is correctly used and called
        base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                     include_top=False,
                                                     weights='imagenet')
        return base_model

    def add_classification_layer(self):
        # Corrected typo in method call (create_base_mode -> create_base_model)
        base_model = self.create_base_model()
        base_model.trainable = False  # Freeze the convolutional base

        # Identify feature batch
        image_batch, label_batch = next(iter(self.train_generator))
        feature_batch = base_model(image_batch)

        # Add GlobalAveragePooling2D and Dense layer for classification
        global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        feature_batch_average = global_average_layer(feature_batch)

        # Apply Dense layer to convert features into a single prediction per image
        prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
        prediction_batch = prediction_layer(feature_batch_average)

        # Create new model by chaining the layers
        preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

        IMG_SIZE = tuple(self.mdict['preprocess']['resize'])

        inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
        x = preprocess_input(inputs)
        x = base_model(inputs, training=False)  # Pass inputs through the base model
        x = global_average_layer(x)  # Apply global average pooling
        x = tf.keras.layers.Dropout(0.2)(x)
        outputs = prediction_layer(x)  # Apply the dense prediction layer

        # Define the complete model
        final_model = tf.keras.Model(inputs, outputs)

        return final_model

    def compile_model(self):
        # Corrected to call add_classification_layer to get the final model
        tf_model = self.add_classification_layer()

        # Fetch the optimizer class using getattr
        learning_rate = self.mdict['transfer_learning']['learning_rate']

        optimizer_class = getattr(tf.keras.optimizers, self.mdict['transfer_learning']['optimizer'])
        loss_class = getattr(tf.keras.losses, self.mdict['transfer_learning']['loss'])
        metrics_class = getattr(tf.keras.metrics, self.mdict['transfer_learning']['metrics']['name'])

        tf_model.compile(loss= loss_class(**{}),
                         optimizer=optimizer_class(learning_rate=df_dict['transfer_learning']['learning_rate']),
                         metrics= [metrics_class(threshold=self.mdict['transfer_learning']['metrics']['params']['threshold'], name=df_dict['transfer_learning']['metrics']['params']['name'])])
        return tf_model

    def fit_model(self):
        # Ensure model is compiled and returned properly
        tf_model = self.compile_model()

        # Fixed indentation and variable reference issues
        log_dir = "../output/logs/" + self.mdict['info']['model_name']
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

        # Ensure correct model and training call
        history = tf_model.fit(
            self.train_generator,  # Train generator
            epochs=self.mdict['transfer_learning']['initial_epochs'],
            validation_data=self.validation_generator,
            callbacks=[tensorboard_callback]
        )

        return history, tf_model  # Return both history and model

    def save_model(self):
        # Ensure model and history are saved correctly
        history, model = self.fit_model()
        model.save(self.mdict['info']['model_filepath'])

        with open(self.mdict['info']['history_filepath'], 'wb') as file:
            pickle.dump(history.history, file)

In [87]:
start_time = time.time()  # Start timing

yaml_file = '../input/base_tf_dict.yaml'

with open(yaml_file, 'r') as file:
    df_dict = yaml.safe_load(file)

generator = Preprocess(**df_dict)
train_generator, validation_generator, test_generator = generator.create_generators()
tf_model = TransferLearning(train_generator, validation_generator, **df_dict)
tf_model.save_model()
end_time = time.time()  # End timing
elapsed_time = end_time - start_time

print(elapsed_time)

Found 800 images belonging to 2 classes.
Found 735 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/10
25/25 [==============================] - 21s 764ms/step - loss: 0.7092 - accuracy: 0.6100 - val_loss: 0.7473 - val_accuracy: 0.4680
Epoch 2/10
25/25 [==============================] - 18s 726ms/step - loss: 0.5917 - accuracy: 0.6975 - val_loss: 0.7911 - val_accuracy: 0.4816
Epoch 3/10
25/25 [==============================] - 18s 717ms/step - loss: 0.5536 - accuracy: 0.7337 - val_loss: 0.8268 - val_accuracy: 0.4857
Epoch 4/10
25/25 [==============================] - 18s 725ms/step - loss: 0.5485 - accuracy: 0.7125 - val_loss: 0.8147 - val_accuracy: 0.4884
Epoch 5/10
25/25 [==============================] - 18s 719ms/step - loss: 0.5217 - accuracy: 0.7425 - val_loss: 0.8895 - val_accuracy: 0.4803
Epoch 6/10
25/25 [==============================] - 18s 713ms/step - loss: 0.5037 - accuracy: 0.7412 - val_loss: 0.8255 - val_accuracy: 0.4803
Epoch 7/10
25/25 [

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


187.99368834495544


# Fine tuning of model

In [101]:
class FineTuning:
    def __init__(self, tf_model, train_generator, validation_generator, **mdict):
      self.mdict = mdict
      self.tf_model = tf_model
      self.train_generator = train_generator
      self.validation_generator = validation_generator

    def identify_ft_layers(self):

      IMG_SIZE = tuple(self.mdict['preprocess']['resize'])
      IMG_SHAPE = IMG_SIZE + (3,)

      self.tf_model.trainable = True
      fine_tune_at = self.mdict['fine_tuning']['fine_tune_at']

      # Freeze all the layers before the `fine_tune_at` layer
      for layer in tf_model.layers[:fine_tune_at]:
        layer.trainable = False

      optimizer_class = getattr(tf.keras.optimizers, self.mdict['fine_tuning']['optimizer'])
      loss_class = getattr(tf.keras.losses, self.mdict['fine_tuning']['loss'])
      metrics_class = getattr(tf.keras.metrics, self.mdict['fine_tuning']['metrics']['name'])

      tf_model.compile(loss= loss_class(**{}),
                    optimizer=optimizer_class(learning_rate=df_dict['fine_tuning']['learning_rate']),
                    metrics= [metrics_class(threshold=self.mdict['fine_tuning']['metrics']['params']['threshold'], name=df_dict['fine_tuning']['metrics']['params']['name'])])

      return tf_model

    def fit_model(self):
      ft_model = self.identify_ft_layers()
      fine_tune_epochs = self.mdict['fine_tuning']['fine_tune_epochs']
      total_epochs = self.mdict['fine_tuning']['fine_tune_epochs'] + self.mdict['transfer_learning']['initial_epochs']

      log_dir = "../output/logs/" + self.mdict['info']['model_name']
      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

      # Ensure correct model and training call
      history = ft_model.fit(
          self.train_generator,  # Train generator
          epochs=total_epochs,
          initial_epoch = self.mdict['transfer_learning']['initial_epochs'],
          validation_data=self.validation_generator,
          callbacks=[tensorboard_callback]
      )

      return history, ft_model

    def save_model(self):
        history, ft_model = self.fit_model()
        ft_model.save(self.mdict['info']['finetune_filepath'])

        with open(self.mdict['info']['finetune_history_filepath'], 'wb') as file:
            pickle.dump(history.history, file)

In [104]:
start_time = time.time()  # Start timing
ft_model = FineTuning(tf_model, train_generator, validation_generator, **df_dict)
ft_model.save_model()
end_time = time.time()  # End timing
elapsed_time = end_time - start_time

print(elapsed_time)

Epoch 11/20
25/25 [==============================] - 20s 756ms/step - loss: 0.4513 - accuracy: 0.7912 - val_loss: 0.8543 - val_accuracy: 0.4816
Epoch 12/20
25/25 [==============================] - 18s 719ms/step - loss: 0.4840 - accuracy: 0.7613 - val_loss: 0.8543 - val_accuracy: 0.4816
Epoch 13/20
25/25 [==============================] - 18s 717ms/step - loss: 0.4645 - accuracy: 0.7800 - val_loss: 0.8543 - val_accuracy: 0.4816
Epoch 14/20
25/25 [==============================] - 18s 721ms/step - loss: 0.4952 - accuracy: 0.7487 - val_loss: 0.8543 - val_accuracy: 0.4816
Epoch 15/20
25/25 [==============================] - 18s 728ms/step - loss: 0.4724 - accuracy: 0.7850 - val_loss: 0.8543 - val_accuracy: 0.4816
Epoch 16/20
25/25 [==============================] - 18s 724ms/step - loss: 0.4693 - accuracy: 0.7825 - val_loss: 0.8543 - val_accuracy: 0.4816
Epoch 17/20
25/25 [==============================] - 18s 735ms/step - loss: 0.4804 - accuracy: 0.7837 - val_loss: 0.8543 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


182.65896105766296
